## Comparing two network states on a diagram with pypowsybl-jupyter
- This notebook demonstrates how to **compare two network snapshots** with the help of the **network explorer widget**.
- It compares **active power flows** (P) on each branches between two network snapshots.
- We will visualize differences by providing the widget a **custom styling** to apply on branches.

#### 1. Loading networks
- We create two network snapshots based on IEEE 300 network.
- We then run an AC loadflow on both snapshots.

In [ ]:
import pypowsybl.network as pn
network1 = pn.create_ieee300()
network2 = pn.create_ieee300()

In [ ]:
network2.update_loads(id='B1-L', p0=140)
network2.update_generators(id='B76-G', target_p=50)

In [ ]:
import pypowsybl.loadflow as lf
lf.run_ac(network1)

In [ ]:
lf.run_ac(network2)

#### 2. The widget!
- Taking a first glance on the network_explorer widget

In [ ]:
from pypowsybl_jupyter import network_explorer
network_explorer(network1, depth=2, vl_id='VL76')

#### 3. Compute branch flow differences
We're constructing the dataframe containing the delta of p1 and p2

- Getting the branches on first snapshot

In [ ]:
branches1 = network1.get_branches(attributes=['p1', 'p2'])
branches1

- Getting the branches on second snapshot

In [ ]:
branches2 = network2.get_branches(attributes=['p1', 'p2'])
branches2

- constructing the `delta_p` dataframe

In [ ]:
import pandas as pd
delta_p = pd.DataFrame()
branches1.fillna(0, inplace=True)
branches2.fillna(0, inplace=True)
delta_p['delta_p1'] = abs(branches2['p1']) - abs(branches1['p1'])
delta_p['delta_p2'] = abs(branches2['p2']) - abs(branches1['p2'])
delta_p

- Looking at the extreme values for `delta_p1`

In [ ]:
delta_p['delta_p1'].min(), delta_p['delta_p1'].max()

In [ ]:
max_row = delta_p.loc[delta_p['delta_p1'].idxmax()]
max_row

In [ ]:
min_row = delta_p.loc[delta_p['delta_p1'].idxmin()]
min_row

#### 4. Construct a style dataframe
  - Green/Red colors indicate negative/positive changes
  - gray indicate values close to 0
  - Line width scales from 1px to 50px with magnitude.

In [ ]:
import numpy as np
import pandas as pd

def create_style_bigradient(n_colors: int = 101) -> pd.DataFrame:
    center = (n_colors - 1) // 2

    colors = np.empty(n_colors, dtype='<U7')
    colors[:center-2] = '#5BD9B3'
    colors[center-2:center+2] = 'gray'
    colors[center+3:] = '#DA4F4C'

    width_neg = np.linspace(50, 1, center)                   # indices 0..49
    width_zero = np.array([1.0])                           # index 50
    width_pos = np.linspace(1, 50, center)                   # indices 51..100
    width_num = np.concatenate([width_neg, width_zero, width_pos])
    width = np.char.add(width_num.astype(int).astype(str), 'px')

    return pd.DataFrame({'hex_color': colors, 'width': width})

def map_index(value: float, n_colors: int)  -> int:
    center = (n_colors - 1) // 2
    v = float(np.clip(value, -50.0, 50.0))
    if v < 0:
        return int(round((center - 1) * (1 - abs(v) / 50.0)))
    if v > 0:
        return int(round(center + (center * (v / 50.0))))
    return center

gradient_size = 101
style = create_style_bigradient(gradient_size)
style

#### 5. Construct the custom style profile

- first taking the default style profile

In [ ]:
default_pf = network1.get_default_nad_profile()

- then overriding the edges (branches) styles based on the delta_p and the style dataframes

In [ ]:
idx = pd.DataFrame({
    'idx_1': delta_p['delta_p1'].apply(lambda x: map_index(x, gradient_size)),
    'idx_2': delta_p['delta_p2'].apply(lambda x: map_index(x, gradient_size)),
})

edges_styles_df = pd.DataFrame({
    'edge1': idx['idx_1'].apply(lambda idx: style['hex_color'].iloc[idx]),
    'edge2': idx['idx_2'].apply(lambda idx: style['hex_color'].iloc[idx]),
    'width1': idx['idx_1'].apply(lambda idx: style['width'].iloc[idx]),
    'width2': idx['idx_2'].apply(lambda idx: style['width'].iloc[idx]),
})
edges_styles_df

- displaying the delta_p on the branches by overriding the branch labels styles

In [ ]:
labels_df = pd.DataFrame({
    'side1': delta_p['delta_p1'].map(lambda x: f"{x:.1f}"),
    'side2': delta_p['delta_p2'].map(lambda x: f"{x:.1f}")
})

- simplifying voltage level descriptions

In [ ]:
vl_descriptions_df = default_pf.vl_descriptions[default_pf.vl_descriptions['type'] != 'FOOTER']

- creating the custom NAD style profile

In [ ]:
diagram_profile=pn.NadProfile(branch_labels=labels_df, vl_descriptions=vl_descriptions_df, bus_descriptions=default_pf.bus_descriptions,
                                      bus_node_styles=default_pf.bus_node_styles, edge_styles=edges_styles_df)

#### 6. Displaying the results!
We can inspect spatial patterns of change on the network_explorer widget

In [ ]:
network_explorer(network1, depth=9, vl_id='VL76', nad_profile=diagram_profile)